In [1]:
import json
import pandas as pd
import requests

In [2]:
follows = pd.read_csv('../shared-folder-gald/data/follow-link.csv')

In [3]:
usernames = pd.unique(follows[['source', 'target']].values.ravel()).tolist()

In [25]:
with open("../shared-folder-gald/keys2.txt") as f:
    lines = f.readlines()
    client_key = lines[0].strip()
    client_secret = lines[1].strip()

In [26]:
def get_access_token(client_key, client_secret):
    # Endpoint URL
    endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"

    # Request headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    # Request body parameters
    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
    }

    # Make the POST request
    response = requests.post(endpoint_url, headers=headers, data=data)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and print the response JSON
        response_json = response.json()
        return response_json['access_token']
    else:
        # If the request was not successful, print the error response JSON
        
        return response.json()

In [27]:
# def get_access_token(client_key, client_secret):
#     endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"
#     headers = {'Content-Type': 'application/x-www-form-urlencoded'}
#     data = {
#         'client_key': client_key,
#         'client_secret': client_secret,
#         'grant_type': 'client_credentials',
#     }

#     response = requests.post(endpoint_url, headers=headers, data=data)
#     response_json = response.json()

#     if response.status_code == 200:
#         print(response_json)  # Debugging print
#         return response_json.get('access_token', 'No access token found')
#     else:
#         print(f"Error {response.status_code}: {response_json}")  # Debugging print
#         return response_json

In [28]:
def get_repost_info(username, access_token,fields ="id, username, favorites_count" ):
    query_params = {"fields": fields}
    query_body = {"username": username}
    headers = {"Authorization": f"Bearer {access_token}"}
    
    endpoint = "https://open.tiktokapis.com/v2/research/user/reposted_videos/"
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)

    if response.status_code == 200:
        # Parse and extract information from the response
        return response.json().get("data", {})
    else:
        
        return {response.text}

In [29]:
access_token = get_access_token(client_key, client_secret)

keeping track with calls
27.02 - we ran keys2 unril 1000 

1.03 - keys2 1001:2002

1.03 - keys1 2002:3003

2.03 - keys1 3003:4004

2.03 - keys2 4004:5005

3.03 - keys1 5005:6006

3.03 - keys2 6006:7007

4.03 - keys1 7007:8008

4.03 - keys2 8008:9009

5.03 - keys1 9009:10009

5.03 - keys2 10009:11009


In [30]:
reposted_videos = []
for username in usernames[10009:11009]:
    reposted_videos.append(get_repost_info(username, access_token))

In [31]:
reposted_videos

[{'cursor': -1, 'has_more': False},
 {'has_more': False, 'cursor': -1},
 {'cursor': -1, 'has_more': False},
 {'has_more': False, 'cursor': -1},
 {'has_more': False,
  'reposted_videos': [{'favorites_count': 699,
    'id': 7442631309375180054,
    'username': 'toutbienpils'},
   {'favorites_count': 4,
    'id': 7470545666721320214,
    'username': 'lepainquotidienhq'}],
  'cursor': -1},
 {'cursor': -1,
  'has_more': False,
  'reposted_videos': [{'favorites_count': 5,
    'id': 7460902418004446506,
    'username': 'johnny_half_acre'},
   {'favorites_count': 1245,
    'id': 7112770088545127686,
    'username': 'musicmoments0'},
   {'favorites_count': 157,
    'id': 7434495913411743008,
    'username': 'lifelikecharlie'},
   {'favorites_count': 133813,
    'id': 7441658882050510110,
    'username': '_false_face'},
   {'id': 7357535146410593578,
    'username': 'randawg3000',
    'favorites_count': 685516},
   {'favorites_count': 57184,
    'id': 7344782904788110634,
    'username': 'ann_an

In [32]:
#saving global reposts for weights in the network
import csv
with open('reposted_videos_raw.csv', 'w') as f:
     
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerow(reposted_videos)

In [33]:
combined = {}
for i, username in enumerate(usernames[10009:11009]):
    combined[username] = reposted_videos[i]
    
combined 


{'wildlanwheels': {'cursor': -1, 'has_more': False},
 'joealbanese0009': {'has_more': False, 'cursor': -1},
 'catpower_official': {'cursor': -1, 'has_more': False},
 'paddlepunk_': {'has_more': False, 'cursor': -1},
 'otovosolar': {'has_more': False,
  'reposted_videos': [{'favorites_count': 699,
    'id': 7442631309375180054,
    'username': 'toutbienpils'},
   {'favorites_count': 4,
    'id': 7470545666721320214,
    'username': 'lepainquotidienhq'}],
  'cursor': -1},
 'handymanuk1': {'cursor': -1,
  'has_more': False,
  'reposted_videos': [{'favorites_count': 5,
    'id': 7460902418004446506,
    'username': 'johnny_half_acre'},
   {'favorites_count': 1245,
    'id': 7112770088545127686,
    'username': 'musicmoments0'},
   {'favorites_count': 157,
    'id': 7434495913411743008,
    'username': 'lifelikecharlie'},
   {'favorites_count': 133813,
    'id': 7441658882050510110,
    'username': '_false_face'},
   {'id': 7357535146410593578,
    'username': 'randawg3000',
    'favorites_

In [34]:
rows = []

for key, value in combined.items():
    if value and 'reposted_videos' in value:  # Ensure key has data
        for video in value['reposted_videos']:
            rows.append({
                'username': key,  # Dictionary key
                'video_id': video['id'],  # Video ID
                'creator': video['username'], # Video username
                'favourites': video["favorites_count"]
            })

# Create DataFrame
reposted_df = pd.DataFrame(rows)

# Display DataFrame
print(reposted_df)

         username             video_id            creator  favourites
0      otovosolar  7442631309375180054       toutbienpils         699
1      otovosolar  7470545666721320214  lepainquotidienhq           4
2     handymanuk1  7460902418004446506   johnny_half_acre           5
3     handymanuk1  7112770088545127686      musicmoments0        1245
4     handymanuk1  7434495913411743008    lifelikecharlie         157
...           ...                  ...                ...         ...
7501    hellomyer  7443293402478906670     harry_and_pals           2
7502    hellomyer  7440469594256444703         kjoftiktok          81
7503    hellomyer  7456472571689602336           __kevocs       39435
7504    hellomyer  7451408837531143466     stevounbelievo         245
7505    hellomyer  7459471247584333099          darrendcl         271

[7506 rows x 4 columns]


In [35]:
reposted_df.to_csv("reposted_videos.csv", mode="a", index=False, header=False)